## Missing data ##

In [2]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [21]:
df = pd.read_csv('/mnt/ffs24/home/lirui15/Research/Model3-growth stage/model3 - all plant-time1.csv')

columns_to_impute = df.columns[221:232] 

mice_imputer = IterativeImputer(max_iter=10, random_state=42)

df[columns_to_impute] = mice_imputer.fit_transform(df[columns_to_impute])

for col in columns_to_impute:
    if (df[col] < 0).any():
        print(f"{col} has negative values.")

for col in columns_to_impute:
    if df[col].isnull().any():
        print(f"{col} has still a vacant value.")
    else:
        print(f"{col}  has no a vacant value.")

output_file_path = '/mnt/ffs24/home/lirui15/Research/Model3-growth stage/model3 - all plant-imputed.csv' 
df.to_csv(output_file_path, index=False)

Molar mass  has no a vacant value.
Additional input  has no a vacant value.
Soil  organic matter (SOM-%)  has no a vacant value.
Soil pH  has no a vacant value.
Cation exchange capacity (CEC-cmol/kg)  has no a vacant value.
Sand (%)  has no a vacant value.
Silt (%)  has no a vacant value.
Clay (%)  has no a vacant value.
Initial concentration in soil  has no a vacant value.
Relative exposure time_1  has no a vacant value.
Relative exposure time_2  has no a vacant value.


## One-hot coding ##

In [22]:
from sklearn.preprocessing import OneHotEncoder

df = pd.read_csv('/mnt/ffs24/home/lirui15/Research/Model3-growth stage/model3 - all plant-imputed.csv')

categorical_columns = df.columns[2:7] 
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_data = encoder.fit_transform(df[categorical_columns])
encoded_columns = encoder.get_feature_names_out(categorical_columns)

encoded_df = pd.DataFrame(encoded_data, columns=encoded_columns)
df.drop(columns=categorical_columns, inplace=True)
df = pd.concat([df, encoded_df], axis=1)

output_file_path = '/mnt/ffs24/home/lirui15/Research/Model3-growth stage/model3 - all plant-encoded.csv'  
df.to_csv(output_file_path, index=False)

## Remove 0-variance column ##

In [23]:
df = pd.read_csv('/mnt/ffs24/home/lirui15/Research/Model3-growth stage/model3 - all plant-encoded.csv')

variances = df.var()
zero_variance_columns = variances[variances == 0].index

df.drop(columns=zero_variance_columns, inplace=True)

print("\nremoved column：")
print(zero_variance_columns)

output_file_path = '/mnt/ffs24/home/lirui15/Research/Model3-growth stage/model3 - all plant-cleand1.csv'
df.to_csv(output_file_path, index=False)


removed column：
Index([], dtype='object')


## Remove high-correlated column ##

In [5]:
df = pd.read_csv('/mnt/ffs24/home/lirui15/Research/Model3-growth stage/model3 - all plant-cleand1-2.csv')

target_column = df.columns[-1]
feature_columns = df.columns[0:228]
#feature_columns = df.columns[0:225].tolist() + [df.columns[226]]

corr_matrix = df[feature_columns].corr(method='spearman').abs()

to_drop = []
high_corr_pairs = []

for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if corr_matrix.iloc[i, j] > 0.8:
            col1 = corr_matrix.columns[i]
            col2 = corr_matrix.columns[j]

            if df[col1].corr(df[target_column]) < df[col2].corr(df[target_column]):
                to_drop.append(col1)
            else:
                to_drop.append(col2)
            high_corr_pairs.append((col1, col2, corr_matrix.iloc[i, j]))

df.drop(columns=to_drop, inplace=True)
print(to_drop)
for pair in high_corr_pairs:
    print(f" {pair[0]} - {pair[1]}, correlation: {pair[2]:.4f}")

df.to_csv('/mnt/ffs24/home/lirui15/Research/Model3-growth stage/model3_reduced1_C_perday.csv', index=False)

['ECFP_2', 'ECFP_38', 'ECFP_18', 'ECFP_18', 'ECFP_51', 'ECFP_34', 'ECFP_29', 'ECFP_34', 'ECFP_66', 'ECFP_54', 'ECFP_18', 'ECFP_51', 'ECFP_63', 'ECFP_29', 'ECFP_74', 'ECFP_13', 'ECFP_125', 'ECFP_125', 'ECFP_125', 'ECFP_2', 'ECFP_33', 'ECFP_29', 'ECFP_74', 'ECFP_109', 'ECFP_148', 'ECFP_148', 'ECFP_148', 'ECFP_29', 'ECFP_74', 'ECFP_151', 'ECFP_151', 'ECFP_29', 'ECFP_74', 'ECFP_153', 'ECFP_153', 'ECFP_151', 'ECFP_159', 'ECFP_159', 'ECFP_159', 'ECFP_148', 'ECFP_34', 'ECFP_66', 'ECFP_94', 'ECFP_125', 'ECFP_34', 'ECFP_66', 'ECFP_94', 'ECFP_125', 'ECFP_167', 'ECFP_175', 'ECFP_34', 'ECFP_66', 'ECFP_94', 'ECFP_125', 'ECFP_167', 'ECFP_172', 'ECFP_2', 'ECFP_33', 'ECFP_135', 'ECFP_148', 'ECFP_159', 'ECFP_18', 'ECFP_51', 'ECFP_63', 'ECFP_107', 'ECFP_34', 'ECFP_66', 'ECFP_94', 'ECFP_125', 'ECFP_167', 'ECFP_172', 'ECFP_176', 'ECFP_205', 'ECFP_205', 'ECFP_208', 'ECFP_39', 'ECFP_161', 'ECFP_104', 'ECFP_29', 'ECFP_74', 'ECFP_240', 'ECFP_240', 'ECFP_151', 'ECFP_153', 'ECFP_196', 'ECFP_34', 'ECFP_66', 'ECF